In [1]:
import newspaper
import csv
import pandas as pd

In [6]:
file = '../../data/training_dataset.csv'

In [3]:
def urls_from_csv(csv_file, column=None):
    '''
    Takes csv directory and returns list of URLs
    '''
    with open(csv_file, 'r') as f:
        reader = csv.reader(f)
        contents = list(reader)
    
    urls = [line[1] for line in contents[1:]]
    return urls, contents

In [4]:
def urls_to_df(csv_file, column=None):
    '''
    Takes csv directory and returns list of URLs
    '''
    df = pd.read_csv(csv_file)
    df.columns = [x.lower() for x in df.columns]
    urls = list(df['url'])
    return urls, df

In [7]:
urls, contents = urls_from_csv(file)

In [8]:
def remove_newline(text):
    ''' Removes new line and &nbsp characters.
    '''
    text = text.replace('\n', ' ')
    text = text.replace('\xa0', ' ')
    return text

In [9]:
urls

['http://www.securitycouncilreport.org/atf/cf/%7B65BFCF9B-6D27-4E9C-8CD3-CF6E4FF96FF9%7D/S_2015_302.pdf',
 'http://www.independent.co.uk/news/world/asia/160-killed-and-hundreds-left-stranded-by-flooding-across-afghanistan-and-pakistan-8746566.html',
 'http://reliefweb.int/sites/reliefweb.int/files/resources/Natural%20Hazards%20Update%204-10%20February_1.pdf',
 'https://www.humanitarianresponse.info/en/operations/afghanistan/document/rafdump01-jan-2015-till-31-dec-2015-0',
 'http://floodlist.com/asia/afghanistan-flash-floods-faryab-baghlan-8-dead',
 'http://floodlist.com/asia/afghanistan-6-dead-flash-floods-kofab-badakhshan-july-2015',
 'http://reliefweb.int/report/afghanistan/afghanistan-earthquake-overview-assessed-needs-and-response-23-november-2015',
 'http://reliefweb.int/sites/reliefweb.int/files/resources/25_dec_15_afg_earthquake_flash_update_-_no._3_-_3_jan_15_0.pdf',
 'http://erccportal.jrc.ec.europa.eu/getdailymap/docId/1125',
 'http://www.unhcr.af/UploadDocs/DocumentLibrary/U

In [10]:
def html_report(link, nlp=False):
    report = {}
    a = newspaper.Article(link)
    a.download()
    a.parse()
    report['domain'] = a.source_url
    report['title'] = a.title
    report['authors'] = a.authors
    report['date_pub'] = a.publish_date
    report['text'] = remove_newline(a.text)
    # tag the type of article
    ## currently default to text but should be able to determine img/video etc
    report['type'] = 'text'
    return report

In [11]:
urls, df = urls_to_df(file)

In [10]:
def scrape_from_urls(urls):
    reports = []
    for url in urls:
        if url[-3:] == 'pdf':
            continue
        else:
            report = html_report(url)
            reports.append(report)
            
    return reports

In [102]:
import requests

In [64]:
from newspaper import Config

In [141]:
config = Config()

In [167]:
user = 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_12_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.81 Safari/537.36'

In [168]:
config.browser_user_agent = user

In [169]:
url = urls.iloc[2]

In [170]:
a = newspaper.Article(url, config)

In [171]:
a.download()

In [174]:
a.title

'Mozambique - Severe Weather (WMO, NOAA, INAM, Local Media) (ECHO Daily Flash, 15 Jan 2016)'

In [172]:
a.parse()

In [173]:
a.text

''

In [95]:
result = requests.get(url)
status = result.status_code

In [96]:
status

200

In [110]:
result.text

'<!DOCTYPE html>\n<html lang="en" dir="ltr" prefix="fb: http://ogp.me/ns/fb# og: http://ogp.me/ns# article: http://ogp.me/ns/article# book: http://ogp.me/ns/book# profile: http://ogp.me/ns/profile# video: http://ogp.me/ns/video# product: http://ogp.me/ns/product#">\n\n<head>\n  <meta http-equiv="Content-Type" content="text/html; charset=utf-8" />\n<meta http-equiv="x-dns-prefetch-control" content="on" />\n<link rel="dns-prefetch" href="//css.static.reliefweb.int" />\n<link rel="dns-prefetch" href="//js.static.reliefweb.int" />\n<link rel="dns-prefetch" href="//img.static.reliefweb.int" />\n<!--[if IE 9]>\n<link rel="prefetch" href="//css.static.reliefweb.int" />\n<link rel="prefetch" href="//js.static.reliefweb.int" />\n<link rel="prefetch" href="//img.static.reliefweb.int" />\n<![endif]-->\n<link rel="shortcut icon" href="http://reliefweb.int/profiles/reliefweb/themes/kobe/favicon.ico" type="image/vnd.microsoft.icon" />\n<link rel="apple-touch-icon" href="/profiles/reliefweb/themes/ko

In [15]:
keys = report[0].keys()
with open('data.csv', 'w') as f:
    dict_writer = csv.DictWriter(f, fieldnames=keys)
    dict_writer.writeheader()
    dict_writer.writerows(report)

NameError: name 'report' is not defined

In [127]:
urls = df_failed['URL']

## Examining the Sources

In [153]:
df_600 = pd.read_csv('../../data/input_urls.csv')

In [155]:
from urllib.parse import urlparse

In [157]:
parsed = urlparse('https://docs.python.org/3/library/urllib.parse.html')

In [161]:
def get_domain(url):
    parsed = urlparse(url)
    domain = parsed.hostname
    return domain

In [162]:
df_600['domain'] = df_600['DocumentIdentifier'].map(lambda x: get_domain(x))

In [179]:
sum(df_600['DocumentIdentifier'].str.contains('pdf'))

30